При необходимости:
*   pip install pymorphy2
*   nltk.download('stopwords')

### Импортируем нужные иструменты

In [21]:
import pandas as pd
import re

from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

### Подгружаем данные

In [2]:
positive = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/positive.csv", sep = ';', header = None)
negative = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/negative.csv", sep = ';', header = None)
full = pd.concat([positive, negative])
full = full.drop(full.columns[[0,1,2,5,6,7,8,9,10,11]], axis = 1)

full.columns=["tweet", "tonality"]

### Предобработка данных

In [4]:
morph = MorphAnalyzer()
ru_stopwords = stopwords.words("russian")

def preprocessing(text):
    text = re.sub("[^А-яа-я]", " ", text)
    result = list()
    for word in text.split():
      if word and word not in ru_stopwords:
        word = morph.parse(word)[0].normal_form
        result.append(word)

    return result

In [5]:
clear_data = full['tweet'].apply(preprocessing)
clear_data = pd.concat([clear_data, full["tonality"]], axis = 1)

### Разделяем на тренировочную и тестовую выборки

In [6]:
train, test = train_test_split(clear_data, test_size = 0.2)
train.columns = ["tweet", "tonality"]
test.columns = ["tweet", "tonality"]
text_train = train['tweet'].apply(lambda x: " ".join(x))
ton_train = train['tonality']
text_test = test['tweet'].apply(lambda x: " ".join(x))
ton_test = test['tonality']

### Логистическая регрессия

In [ ]:
logreg = LogisticRegression(solver = 'newton-cg')
score = [0, 0, 0, 0]

### Н-граммы для BOW

In [19]:
for i in [1,2,3]:
  vectorizer = CountVectorizer(ngram_range = (i, i))
  text_train_ngram = vectorizer.fit_transform(text_train)
  text_test_ngram = vectorizer.transform(text_test)
  logreg.fit(text_train_ngram, ton_train)
  lr_prediction = logreg.predict(text_test_ngram)
  score[i] = f1_score(ton_test, lr_prediction)

### TF-IDF

In [22]:
vectorizer = TfidfVectorizer()
text_train_tfidf = vectorizer.fit_transform(text_train)
text_test_tfidf = vectorizer.transform(text_test)
logreg.fit(text_train_tfidf, ton_train)
lr_prediction = logreg.predict(text_test_tfidf)
score[0] = f1_score(ton_test, lr_prediction)

### Результаты

In [38]:
print('TF-IDF\t\t\t1Grams\t\t\t2Grams\t\t\t3Grams')
print(score)

TF-IDF			1Grams			2Grams			3Grams
[0.7357669193688816, 0.7312112407303005, 0.6880903573294747, 0.2855409648872395]
